# Clustering

The clustering solution tries to address the problem:

*Mixing of personal and business expenses is a common problem for small business. Create an algorithm that can separate any potential personal expenses in the training data. Labels of personal and business expenses were deliberately not given as this is often the case in our system. There is no right answer so it is important you provide any assumptions you have made.*

For this, we will use K-means clustering for simplicity.

For distinguishing personal expenses from business expenses, we are going to make the following **assumptions** (**none of those are necessarily true in all cases**):

- If the date of expenses is on weekeends or holidays, it is more likely to be a personal expense (notice, however, the flaw of this assumption is that an expense on Friday night might very well be personal)
- Higher expense amount has more chance of being a business expense
- The text fields along with the previous 2 criteria (expense description and category) can possibly give us some clue about the type of expense. For example, category **office supplies** and **Computer Hardware** has more chance of being a business expense, whereas **Meals and Entertainment** with a relatively low expense amount has a higher chance of being a personal expense, especially if it is on a weekend.
- Employee ID might useful as well since certain employees might have certain habits of making personal or business type expenses.
- Same goes for role.

(Though not shown in this notebook, later I have found that excluding employee id and role does not alter the resulting clusters)

# Necessary imports

In [157]:
import pandas as pd
import numpy as np
from collections import defaultdict
import itertools

from sklearn import preprocessing
from sklearn.cluster import KMeans

Read in the data

In [176]:
df_train = pd.read_csv('training_data_example.csv')
df_val = pd.read_csv('validation_data_example.csv')
df_employee = pd.read_csv('employee.csv')

# Ready the features to be used

We will define a pre-processing function to ready our features. 
- It has an internal function to determine whether a prticular day is a holiday. It's kind of simplified and by no means an exact dertminer of a holiday. I am assuming weekends and any day after December 19 as a holiday.
- It basically drops all columns except employee id, role and pre-tax amount
- Text features (category and expense description will be processed separately)
- Pre-tax amount is normalized to 0-1 scale

In [239]:
def pre_process(df, columns_to_drop=['date',
                                     'employee id', 
                                     'category', 
#                                    'pre-tax amount', 
                                     'tax amount',
                                     'tax name',  
                                     'role', 
                                     'expense description']):
    
    def holiday(x):
        if x.weekday() >= 5:
            return 1
        if x.month == 12 and x.day >= 20:
            return 1
        return 0
    
    df['holiday'] = pd.to_datetime(df['date']).apply(lambda x: holiday(x)).astype(str)    
    df = pd.merge(df, df_employee[['employee id', 'role']], how='inner', on=['employee id'])
    df['employee id'] = df['employee id'].astype(str)
    df = df.drop(columns_to_drop, axis=1)
    
    # one-hot encode the categorical variables
    df = pd.get_dummies(df)
    
    df['pre-tax amount'] = preprocessing.minmax_scale(df[['pre-tax amount']])
    
    return df

Get the pre-process features for training and validation data.

In [240]:
x_train = pre_process(df_train)
x_val = pre_process(df_val)
x_train, x_val = x_train.align(x_val, join='left', axis=1)
x_val = x_val.fillna(0)

In [241]:
x_train.head()

,pre-tax amount,holiday_0,holiday_1,text_data_Airplane ticket to NY Travel,text_data_Client dinner Meals and Entertainment,text_data_Coffee with Steve Meals and Entertainment,text_data_Dinner Meals and Entertainment,text_data_Dinner with client Meals and Entertainment,text_data_Dinner with potential client Meals and Entertainment,text_data_Dropbox Subscription Computer - Software,text_data_Flight to Miami Travel,text_data_HP Laptop Computer Computer - Hardware,text_data_Macbook Air Computer Computer - Hardware,text_data_Microsoft Office Computer - Software,text_data_Paper Office Supplies,text_data_Starbucks coffee Meals and Entertainment,text_data_Taxi ride Travel,text_data_Team lunch Meals and Entertainment,text_data_iCloud Subscription Computer - Software,text_data_iPhone Computer - Hardware
0,0.018045,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0.098246,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.000000,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0.115789,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0.005514,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


# Processing the text fields

For this purpose, we can make use of as much text as available.

- We will concatenate **category** and **text description** together into a single field
- Since K-means will internally use Euclidean distance, we will vecorize the field. We will try 2 different ways if vectorizing it:
    - Using Scikit learn's TfIdfVectoirizer (which is just a frequency based embedding of the words)
    - Using a pre-trained prediction based embedding of the words. This approach makes more sense because this sort of embeddings caoture useful relations between words.  We will use the **glove** embedding https://nlp.stanford.edu/projects/glove/ with 100 dimensional vector. We will use the technique describe here http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/. The following class was taken from this reference. I tweaked it slightly to work with python 3.
    - We will then compare the results of both of these approaches

In [179]:
# Reference: http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.values())

    def fit(self, X):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

First, we will read in the embedding vectors from the relevant file and fit using the above TfidfEmbeddingVectorizer

In [152]:
with open("glove.6B.100d.txt", "rb") as lines:
    w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

We will create a new field concatenating category and expense description.

In [242]:
df_train['text_data'] = df_train['expense description'] + ' ' + df_train['category']
df_val['text_data'] = df_val['expense description'] + ' ' + df_val['category']

Let's fit the embedding vectorizer, which considers TfIdf on top of the embedding vectors.

In [243]:
embedding_vectorizer = TfidfEmbeddingVectorizer(w2v)
embedding_vectorizer.fit(df_train['text_data'])

Let's also fit using only the TfIdfVectorizer.

In [244]:
vectorizer = TfidfVectorizer(stop_words='english')
vectorizer.fit(df_train['text_data'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

Let's prepare the text vectors for the training and validation data, for both - using and not using pre-trained embeddings.

In [245]:
x_train_tfidf = vectorizer.transform(df_train['text_data']).toarray()
x_val_tfidf = vectorizer.transform(df_val['text_data']).toarray()

x_train_tfidf_embedding = embedding_vectorizer.transform(df_train['text_data'])
x_val_tfidf_embedding = embedding_vectorizer.transform(df_val['text_data'])

Concatenate the text vectors with the non-text feature vectors to get the final feature vectors (again, for both - using and not using pre-trained embeddings.)

In [246]:
x_train_final_no_embedding = np.concatenate((x_train.values, x_train_tfidf), axis=1)
x_val_final_no_embedding = np.concatenate((x_val.values, x_val_tfidf), axis=1)

x_train_final_embedding = np.concatenate((x_train.values, x_train_tfidf_embedding), axis=1)
x_val_final_embedding = np.concatenate((x_val.values, x_val_tfidf_embedding), axis=1)

## Apply K-means

Now that we have the fetures ready, we will apply K-means to it.
First, let's try with the one that has no pretrained embeddings considered (only tf-idf).

In [261]:
kmeans = KMeans(n_clusters=2, random_state=1)
kmeans.fit(x_train_final_no_embedding)
clusters_train = kmeans.predict(x_train_final_no_embedding)

In [248]:
clusters_train

array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1], dtype=int32)

Let's have a look at the clusters.

In [249]:
cluster1 = df_train[clusters_train==0]
cluster2 = df_train[clusters_train==1]

In [250]:
cluster1

,date,category,employee id,expense description,pre-tax amount,tax name,tax amount,holiday,text_data
0,11/1/2016,Travel,7,Taxi ride,40.0,NY Sales tax,3.55,0,Taxi ride Travel
1,11/15/2016,Meals and Entertainment,1,Team lunch,235.0,CA Sales tax,30.55,0,Team lunch Meals and Entertainment
2,11/30/2016,Computer - Hardware,3,HP Laptop Computer,999.0,CA Sales tax,129.87,0,HP Laptop Computer Computer - Hardware
3,11/14/2016,Computer - Software,3,Microsoft Office,899.0,CA Sales tax,116.87,0,Microsoft Office Computer - Software
6,12/9/2016,Meals and Entertainment,6,Coffee with Steve,300.0,CA Sales tax,39.00,0,Coffee with Steve Meals and Entertainment
7,11/12/2016,Travel,4,Taxi ride,230.0,CA Sales tax,29.90,1,Taxi ride Travel
8,11/21/2016,Meals and Entertainment,7,Client dinner,200.0,NY Sales tax,17.75,0,Client dinner Meals and Entertainment
9,10/4/2016,Travel,6,Flight to Miami,200.0,CA Sales tax,26.00,0,Flight to Miami Travel
10,10/12/2016,Computer - Hardware,7,Macbook Air Computer,1999.0,NY Sales tax,177.41,0,Macbook Air Computer Computer - Hardware
14,12/30/2016,Meals and Entertainment,4,Dinner with potential client,200.0,CA Sales tax,26.00,1,Dinner with potential client Meals and Enterta...


In [251]:
cluster2

,date,category,employee id,expense description,pre-tax amount,tax name,tax amount,holiday,text_data
4,11/6/2016,Computer - Software,4,Dropbox Subscription,50.0,CA Sales tax,6.50,1,Dropbox Subscription Computer - Software
5,11/3/2016,Computer - Software,3,Dropbox Subscription,50.0,CA Sales tax,6.50,0,Dropbox Subscription Computer - Software
11,12/11/2016,Computer - Software,1,iCloud Subscription,15.0,CA Sales tax,1.95,1,iCloud Subscription Computer - Software
12,9/18/2016,Travel,1,Taxi ride,60.0,CA Sales tax,7.80,1,Taxi ride Travel
13,9/30/2016,Office Supplies,3,Paper,200.0,CA Sales tax,26.00,0,Paper Office Supplies
17,12/3/2016,Meals and Entertainment,5,Starbucks coffee,4.0,CA Sales tax,0.52,1,Starbucks coffee Meals and Entertainment
21,12/8/2016,Meals and Entertainment,4,Dinner,180.0,CA Sales tax,23.40,0,Dinner Meals and Entertainment
22,12/31/2016,Meals and Entertainment,4,Dinner,1000.0,CA Sales tax,130.00,1,Dinner Meals and Entertainment
23,12/9/2016,Meals and Entertainment,4,Dinner,30.0,CA Sales tax,3.90,0,Dinner Meals and Entertainment


## Observations:

- One cluster has 15 data ponits, whereas the other has 9
- The first one has fewer holidays, all the computer hardware stuffs and meals and entertainment that re mostly related to client, team etc, indicating that these are business expense.
- The seond cluster has more holidays, meals and entertainment that look more personal, even though it has the **Office Supplies** record which clearly in the wrong cluster.
- Overall the clusters kind of make sense.

Now let's try with pre-trained embedding vectors.

In [268]:
kmeans = KMeans(n_clusters=2, random_state=1)
kmeans.fit(x_train_final_embedding)
clusters_train = kmeans.predict(x_train_final_embedding)

In [253]:
clusters_train

array([1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0], dtype=int32)

In [254]:
cluster1 = df_train[clusters_train==0]
cluster2 = df_train[clusters_train==1]

In [255]:
cluster1

,date,category,employee id,expense description,pre-tax amount,tax name,tax amount,holiday,text_data
4,11/6/2016,Computer - Software,4,Dropbox Subscription,50.0,CA Sales tax,6.50,1,Dropbox Subscription Computer - Software
5,11/3/2016,Computer - Software,3,Dropbox Subscription,50.0,CA Sales tax,6.50,0,Dropbox Subscription Computer - Software
11,12/11/2016,Computer - Software,1,iCloud Subscription,15.0,CA Sales tax,1.95,1,iCloud Subscription Computer - Software
12,9/18/2016,Travel,1,Taxi ride,60.0,CA Sales tax,7.80,1,Taxi ride Travel
13,9/30/2016,Office Supplies,3,Paper,200.0,CA Sales tax,26.00,0,Paper Office Supplies
17,12/3/2016,Meals and Entertainment,5,Starbucks coffee,4.0,CA Sales tax,0.52,1,Starbucks coffee Meals and Entertainment
21,12/8/2016,Meals and Entertainment,4,Dinner,180.0,CA Sales tax,23.40,0,Dinner Meals and Entertainment
22,12/31/2016,Meals and Entertainment,4,Dinner,1000.0,CA Sales tax,130.00,1,Dinner Meals and Entertainment
23,12/9/2016,Meals and Entertainment,4,Dinner,30.0,CA Sales tax,3.90,0,Dinner Meals and Entertainment


In [256]:
cluster2

,date,category,employee id,expense description,pre-tax amount,tax name,tax amount,holiday,text_data
0,11/1/2016,Travel,7,Taxi ride,40.0,NY Sales tax,3.55,0,Taxi ride Travel
1,11/15/2016,Meals and Entertainment,1,Team lunch,235.0,CA Sales tax,30.55,0,Team lunch Meals and Entertainment
2,11/30/2016,Computer - Hardware,3,HP Laptop Computer,999.0,CA Sales tax,129.87,0,HP Laptop Computer Computer - Hardware
3,11/14/2016,Computer - Software,3,Microsoft Office,899.0,CA Sales tax,116.87,0,Microsoft Office Computer - Software
6,12/9/2016,Meals and Entertainment,6,Coffee with Steve,300.0,CA Sales tax,39.00,0,Coffee with Steve Meals and Entertainment
7,11/12/2016,Travel,4,Taxi ride,230.0,CA Sales tax,29.90,1,Taxi ride Travel
8,11/21/2016,Meals and Entertainment,7,Client dinner,200.0,NY Sales tax,17.75,0,Client dinner Meals and Entertainment
9,10/4/2016,Travel,6,Flight to Miami,200.0,CA Sales tax,26.00,0,Flight to Miami Travel
10,10/12/2016,Computer - Hardware,7,Macbook Air Computer,1999.0,NY Sales tax,177.41,0,Macbook Air Computer Computer - Hardware
14,12/30/2016,Meals and Entertainment,4,Dinner with potential client,200.0,CA Sales tax,26.00,1,Dinner with potential client Meals and Enterta...


## Observations:

It turns out that we got the exact same clusters. So using pre-trained embeddings hasn't really helped too much in this case, may be because there wasn't enough data for the embeddings to take effect. Nevertheless, it was a good exercise for me to go through both approaches.

## Just try the performance on the validation set

Although not asked in the question, I am just trying out the performace on the validation set as well using the one that uses pre-trained embeddings.

In [269]:
clusters_val = kmeans.predict(x_val_final_embedding)

In [270]:
cluster1 = df_val[clusters_val==0]
cluster2 = df_val[clusters_val==1]

In [271]:
cluster1

,date,category,employee id,expense description,pre-tax amount,tax name,tax amount,holiday,text_data
1,11/12/2016,Meals and Entertainment,1,Dinner with Family,235.0,CA Sales tax,30.55,1,Dinner with Family Meals and Entertainment
9,12/2/2016,Meals and Entertainment,4,Dinner,180.0,CA Sales tax,23.40,0,Dinner Meals and Entertainment
10,2/14/2017,Meals and Entertainment,4,Dinner,500.0,CA Sales tax,65.00,0,Dinner Meals and Entertainment


In [272]:
cluster2

,date,category,employee id,expense description,pre-tax amount,tax name,tax amount,holiday,text_data
0,11/10/2016,Travel,7,Taxi ride,30.0,NY Sales tax,2.66,0,Taxi ride Travel
2,9/2/2016,Computer - Hardware,4,Macbook Air Computer,4000.0,CA Sales tax,520.00,0,Macbook Air Computer Computer - Hardware
3,9/2/2016,Office Supplies,4,Paper,20.0,CA Sales tax,2.60,0,Paper Office Supplies
4,9/2/2016,Office Supplies,4,Pens,20.0,CA Sales tax,2.60,0,Pens Office Supplies
5,11/21/2016,Travel,1,Airplane ticket to Miami,200.0,CA Sales tax,26.00,0,Airplane ticket to Miami Travel
6,12/4/2016,Meals and Entertainment,2,Starbucks coffee,12.0,CA Sales tax,1.56,1,Starbucks coffee Meals and Entertainment
7,1/18/2010,Meals and Entertainment,6,Dinner,30.0,CA Sales tax,3.90,0,Dinner Meals and Entertainment
8,10/5/2016,Meals and Entertainment,4,Dinner with client,220.0,CA Sales tax,28.60,0,Dinner with client Meals and Entertainment
11,2/14/2016,Meals and Entertainment,4,Dinner,600.0,CA Sales tax,78.00,1,Dinner Meals and Entertainment


## Observations:

It seems that the performance on the validation set is not bad after all. Thefirst cluster hints mostly personal, whereas the second cluster hints mostly business expenses. Note that it has now put **Office Supplies** rightly in the sond cluster.

## Conclusion:

There can be a myria of other ways we can possibly try to do the clustering. I have tried as much as I can in my limited time. A bit of improvement on my approach could be that we could try mutiple iterations of the K-means fitting using mutiple initial random state, and then then assign each data point to the cluster that they belong to the most number of times.